In [ ]:
from PRM2ORM import PRM2ORM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


# initialize variable
INPUT_PATH = 'src/en-zh'
# The name/path of the trained model and reference model.
TRAINED_PATH = 'sabijun/MT-PRM-LLaMA-3.2-3B'
REF_PATH = 'meta-llama/Llama-3.2-3B-Instruct'
# gpu index
trained_device = 'cuda:0'
ref_device = 'cuda:1'
# The method of Sequence-level score calculation.
# Possible choices: 'weighted', 'length normalized'
method = 'weighted'
# correspond with the translation direction
# e.g. from Chinese to English
language_pair = ['English', 'Chinese']

In [2]:
# load the models
trained_model = AutoModelForCausalLM.from_pretrained(TRAINED_PATH, torch_dtype=torch.bfloat16).to(trained_device)
ref_model = AutoModelForCausalLM.from_pretrained(REF_PATH, torch_dtype=torch.bfloat16).to(ref_device)
tokenizer = AutoTokenizer.from_pretrained(REF_PATH)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
prm2orm = PRM2ORM(trained_model, trained_device, ref_model, ref_device, tokenizer)

In [6]:
# read the sentence
f = open(INPUT_PATH, 'r', encoding='utf-8')
lines = f.read().splitlines()
f.close()

f = open('output/results', 'w', encoding='utf-8')
for i in range(len(lines) // 2):
    score = prm2orm.score(['English', 'Chinese'], lines[2*i], lines[2*i+1], method)
    f.write(lines[2*i] + '\n' + lines[2*i+1] + '\t' + str(score.item()) + '\n')
    print(lines[2*i] + '\n' + lines[2*i+1] + '\nscore:' + str(score.item()))
f.close()

/home/jiahan/anaconda3/envs/mt/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/jiahan/anaconda3/envs/mt/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/jiahan/anaconda3/envs/mt/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
From v4.47 onwards, when a model cache is

The hacked up version of Jedi Knight was crashing because it was calling a function off the end of a vtable.
绝地武士的破解版崩溃了，因为它调用了 vtable 结尾的一个函数。
score:-3.2380893230438232
Even if it's true that such facts exist in science it's still possible to argue that scientific facts are theory-laden.
即使这些事实确实存在于科学之中，仍然有可能认为科学事实是纯理论的。
score:2.5001437664031982
